In [ ]:
from glob import glob
import sys
import numpy as np
import pandas as pd
from tqdm import tqdm
from copy import copy
from matplotlib import pyplot as plt
%matplotlib inline
from matplotlib.colors import LogNorm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
import helper
traj = helper.build_angres_trajectory()

In [ ]:
from neutrino_level.steps.general_modules.read_data import load_dfs
mc_df, data_df = load_dfs(
    traj, ignore_tables=helper.to_ignore)

In [ ]:
def get_opening_angle(zen0, azi0, zen1, azi1):
    angles = np.zeros_like(zen0)
    angles += np.sin(zen1) * np.sin(zen0)
    angles *= np.cos(azi1 - azi0)
    angles += np.cos(zen1) * np.cos(zen0)
    angles = np.clip(angles, -1., 1.)
    angles = np.arccos(angles)
    return angles

In [ ]:
angles = get_opening_angle(
    mc_df['MCPrimary.zenith'],
    mc_df['MCPrimary.azimuth'],
    mc_df['EHE_SplineMPE.zenith'],
    mc_df['EHE_SplineMPE.azimuth'],
)
mc_df['opening_angle'] = angles
mc_df['opening_angle_in_deg'] = angles * 180 / np.pi

angles = get_opening_angle(
    mc_df['MCPrimary.zenith'],
    mc_df['MCPrimary.azimuth'],
    mc_df['EHELineFit.zenith'],
    mc_df['EHELineFit.azimuth'],
)
mc_df['opening_angle_lf'] = angles
mc_df['opening_angle_lf_in_deg'] = angles * 180 / np.pi

angles = get_opening_angle(
    mc_df['EHE_SplineMPE.zenith'],
    mc_df['EHE_SplineMPE.azimuth'],
    mc_df['EHELineFit.zenith'],
    mc_df['EHELineFit.azimuth'],
)
mc_df['opening_angle_recos'] = angles
mc_df['opening_angle_recos_in_deg'] = angles * 180 / np.pi

In [ ]:
np.savez('angres_comparison.npz',
         open_angle_deg_new=mc_df['opening_angle_in_deg'],
         open_angle_deg_old=mc_df['opening_angle_lf_in_deg']
         )

In [ ]:
weights = None
bins = np.linspace(-1, 10, 100)
fig, ax = plt.subplots()
ax.hist(mc_df['opening_angle_in_deg'],
        bins=bins,
        histtype='step',
        lw=2,
        label='Spline MPE',
        weights=weights,
        density=True
        )
ax.hist(mc_df['opening_angle_lf_in_deg'],
        bins=bins,
        histtype='step',
        lw=2,
        label='LineFit',
        weights=weights,
        density=True
        )
print(np.median(mc_df['opening_angle_in_deg']))
print(np.median(mc_df['opening_angle_lf_in_deg']))
ax.set_xlim([-0.5,5])
# ax.set_yscale('log')
ax.legend()
ax.set_xlabel('Opening angle / deg')
ax.set_ylabel('Entires / bin')
#fig.savefig('figs_l4v2/opening_angle_comparison.pdf', bbox_inches='tight')